<a href="https://colab.research.google.com/github/mofuoku417/anomaly-detection/blob/main/Detecting_Anomalies_In_Natural_Gas_Prices_Using_Deep_Learning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

###**Problem statement**

This project aims to analyze crude oil data from 01/04/200 - 6/17/22 using anomaly detection techniques to uncover hidden patterns and outliers in the data. We will use the Pytorch library in Python to construct and train a model to detect anomalous data points within the dataset. Finally, we will analyze and interpret our results to draw meaningful conclusions about the Crude oil market.

In [2]:
!nvidia-smi

Mon Jul 31 03:17:30 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.105.17   Driver Version: 525.105.17   CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   53C    P8    12W /  70W |      0MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [3]:
!pip install chart_studio

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.4/64.4 kB 2.1 MB/s eta 0:00:00


In [4]:
!pip install plotly==4.1.1

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.1/7.1 MB 17.0 MB/s eta 0:00:00
  Attempting uninstall: plotly
    Found existing installation: plotly 5.13.1
    Uninstalling plotly-5.13.1:
      Successfully uninstalled plotly-5.13.1


In [5]:
# Check for GPU
import torch
torch.cuda.is_available()

True

####Loading the required libraries

In [ ]:
from pandas import read_csv

import plotly
#import plotly.plotly as py
import chart_studio.plotly as py
import cufflinks as cf
from plotly.offline import init_notebook_mode, iplot
init_notebook_mode(connected=True)
import plotly.graph_objs as go
import plotly.offline as offline
offline.init_notebook_mode()

from matplotlib import pyplot as plt
import seaborn as sns

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import torch.nn as nn
from datetime import date, timedelta # Date Operations
import matplotlib.pyplot as plt # To aid in visualizing
%matplotlib inline
import seaborn as sns
import matplotlib.dates as mdates # Date formatting

####Loading the dataset

In [7]:
# Establishing a timetable for data extraction
today = date.today()
end_date = today.strftime("%Y-%m")
#start_date = '1993-03'
#start_date = '07-01-97'
start_date = '1997-01-07'

featurename = 'Henry Hub Daily Oil Price'

#Load dataset
path ='https://raw.githubusercontent.com/mofuoku417/anomaly-detection/main/Data/HHD.csv'
df = read_csv(path, parse_dates=True)

# Henry Hub Natural Gas Spot Price (I:HHNGSP)
# 2.66 USD/MMBtu for Jul 25 2023
#
# "%Y-%m"
# "%Y-%m-%d"
# '%d-%m-%Y'
# ('%d-%m-%Y')

In [8]:
path ='https://raw.githubusercontent.com/mofuoku417/anomaly-detection/main/Data/HHM.csv'
dataframe = read_csv(path, parse_dates=True,)

In [9]:
df.dtypes

Date     object
HHD     float64
dtype: object

###Data Preprocessing

In [10]:
print(f"Downloaded {len(df)} rows of data.")
df

Downloaded 6676 rows of data.


,Date,HHD
0,1997-01-07,3.82
1,1997-01-08,3.80
2,1997-01-09,3.61
3,1997-01-10,3.92
4,1997-01-13,4.00
...,...,...
6671,2023-07-19,2.51
6672,2023-07-20,2.61
6673,2023-07-21,2.61
6674,2023-07-24,2.68


In [11]:
#Checking the data types
df.dtypes

Date     object
HHD     float64
dtype: object

In [12]:
device = 'cuda:0' if torch.cuda.is_available() else 'cpu'
device

'cuda:0'

In [13]:
# Converting date column from object to date format
df['Date'] = pd.to_datetime(df['Date'].apply(lambda x: x.split()[0])) # Selecting only date
#df['Date'] = pd.to_datetime(df['Date'].apply(lambda x: x.split()[0])) # Selecting only date

In [16]:
import plotly.graph_objects as go
import plotly.graph_objs as go
import plotly.express as px
px.line(df, x = 'Date', y = 'HHD', title="Natural gas").update_layout(
    xaxis_title="Date", yaxis_title="7 day avg")
fig.update_layout(xaxis_title = "Date_time", yaxis_title = "Price ($/Mbtu)",
                  title_text="Henry Hub Spot Price (Daily frequency)")
fig.show(renderer="colab")

# fig1.layout.update()

# fig.update_layout(xaxis_title = "Date_time", yaxis_title = "Price ($/Mbtu)",
                  #title_text="Henry Hub Spot Price (Daily frequency)")

NameError: ignored

### Display the Timeseries

### Detect Missing Data

In [17]:
print('The Original DataFrame:')
print(df)
print('\n')

The Original DataFrame:
           Date   HHD
0    1997-01-07  3.82
1    1997-01-08  3.80
2    1997-01-09  3.61
3    1997-01-10  3.92
4    1997-01-13  4.00
...         ...   ...
6671 2023-07-19  2.51
6672 2023-07-20  2.61
6673 2023-07-21  2.61
6674 2023-07-24  2.68
6675 2023-07-25  2.66

[6676 rows x 2 columns]




In [18]:
# Count Missing Values in DataFrame
# Filling missing values using fillna(), replace() and interpolate()....Detect missing values with isnull() and isna()

print(df.isnull())

       Date    HHD
0     False  False
1     False  False
2     False  False
3     False  False
4     False  False
...     ...    ...
6671  False  False
6672  False  False
6673  False  False
6674  False  False
6675  False  False

[6676 rows x 2 columns]


In [19]:
print(df.notnull())

      Date   HHD
0     True  True
1     True  True
2     True  True
3     True  True
4     True  True
...    ...   ...
6671  True  True
6672  True  True
6673  True  True
6674  True  True
6675  True  True

[6676 rows x 2 columns]


### Count Missing Values in Each rRow and Column

In [20]:
print(df.isnull().sum())

Date    0
HHD     1
dtype: int64


In [21]:
print(df.isnull().values.sum())

1


### Drop the Row with Missing Value

In [22]:
# By default, we drop rows with at least one missing value
print('The DataFrame after removing the rows with empty fields')
print(df.dropna())

The DataFrame after removing the rows with empty fields
           Date   HHD
0    1997-01-07  3.82
1    1997-01-08  3.80
2    1997-01-09  3.61
3    1997-01-10  3.92
4    1997-01-13  4.00
...         ...   ...
6671 2023-07-19  2.51
6672 2023-07-20  2.61
6673 2023-07-21  2.61
6674 2023-07-24  2.68
6675 2023-07-25  2.66

[6675 rows x 2 columns]


Deviation from established pattern is considered an anomaly in a given data. Apply a Boxplot to visualize these deviations.

In [26]:
fig = px.box(
    data_frame = df
    ,y = 'HHD'
)
fig.update_layout(
    autosize=False,
    width=1000,
    height=600
)

fig.show(renderer="colab")